In [6]:
#0. install any required dependences
!pip install flask networkx plotly
!python -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [7]:
project_code = '''\
# 1. Initializing Libraries
from flask import Flask, render_template, request, redirect, url_for, jsonify
import json
import os
import networkx as nx
import time
import plotly.graph_objects as go
from datetime import datetime, timedelta
import random


#Global variables
undo_stack = []
last_graph_generation_time = 0

# 2. Flask App Setup
app = Flask(__name__)
def background_task():
    # Simulate a long-running task
    time.sleep(2)
    print("Background task finished!")

def initialize_folder_structure():
    # Create folders if they don't exist
    if not os.path.exists("static"):
        os.makedirs("static")
    if not os.path.exists("templates"):
        os.makedirs("templates")

    # Create a basic CSS file if it doesn't exist
    if not os.path.exists("static/styles.css"):
        with open("static/styles.css", "w") as f:
            f.write("""\
body {
    font-family: Arial, sans-serif;
    padding: 20px;
    background-color: #f4f4f4; /* Light mode background */
    color: #333; /* Light mode text color */
    transition: background-color 0.3s, color 0.3s; /* Smooth transition between modes */
}

body.dark-mode {
    background-color: #333; /* Dark mode background */
    color: #f4f4f4; /* Dark mode text color */
}

h1, header nav {
    text-align: center;
}

table {
    width: 100%;
    border-collapse: collapse;
}

th, td {
    padding: 10px;
    text-align: left;
    border: 1px solid #ddd;
}

body.dark-mode th, body.dark-mode td {
    border: 1px solid #555; /* Darker borders for dark mode */
    background-color: #444; /* Dark mode table rows */
    color: #f4f4f4;
}

a {
    text-decoration: none;
    color: #007bff;
}

body.dark-mode a {
    color: #66aaff; /* Lighter blue for links in dark mode */
}

form input, form select {
    margin-bottom: 15px;
    padding: 8px;
    width: 100%; /* Ensure consistent width for all form inputs */
    max-width: 300px; /* Limit input width for better design */
}

form label {
    display: block;
    margin-bottom: 5px; /* Align labels consistently */
}

form button {
    width: auto; /* Adjust button width to content */
}

#loading {
    display: none;
    text-align: center;
    margin-top: 20px;
    font-weight: bold;
    color: #555;
}

nav {
    margin-bottom: 20px;
}

button {
    background-color: #007bff;
    color: white;
    border: none;
    padding: 10px;
    cursor: pointer;
}

button:hover {
    background-color: #0056b3;
}

body.dark-mode button {
    background-color: #0056b3; /* Darker button in dark mode */
}

body.dark-mode button:hover {
    background-color: #003f7f; /* Darker hover effect */
}

#toggleButton {
    position: fixed;
    top: 20px;
    right: 20px;
    padding: 10px 20px;
    cursor: pointer;
    background-color: #333;
    color: white;
    border: none;
    border-radius: 5px;
}

body.dark-mode #toggleButton {
    background-color: #f4f4f4;
    color: #333;
}
""")

    # Create a basic JavaScript file if it doesn't exist
    if not os.path.exists("static/scripts.js"):
        with open("static/scripts.js", "w") as f:
            f.write("""\
                // Function to toggle dark mode on and off
                function toggleDarkMode() {
                    // Toggle the dark-mode class on the body element
                    document.body.classList.toggle('dark-mode');

                    // Save the current theme preference in localStorage
                    if (document.body.classList.contains('dark-mode')) {
                        localStorage.setItem('theme', 'dark');
                    } else {
                        localStorage.setItem('theme', 'light');
                    }
                }

                // Automatically reformat dates in DD/MM/YYYY format
                document.addEventListener('DOMContentLoaded', function () {
                    document.querySelectorAll('input[type="date"]').forEach(function (input) {
                        input.addEventListener('change', function () {
                            const dateValue = new Date(this.value);
                            const day = String(dateValue.getDate()).padStart(2, '0');
                            const month = String(dateValue.getMonth() + 1).padStart(2, '0');
                            const year = dateValue.getFullYear();
                            this.value = `${day}/${month}/${year}`;
                        });
                    });
                });

                // Check if a theme is saved in localStorage and apply it on page load
                window.onload = function() {
                    // Get the saved theme from localStorage
                    const savedTheme = localStorage.getItem('theme');

                    // If there's a saved theme, apply it
                    if (savedTheme === 'dark') {
                        document.body.classList.add('dark-mode');
                    } else {
                        document.body.classList.remove('dark-mode');
                    }
                }

                // Event listener for the toggle button
                document.getElementById('toggleButton').addEventListener('click', toggleDarkMode);
                    
                // Updating forms
                function updateCategoryFields() {
                    const category = document.getElementById("category").value.toLowerCase();
                    const fields = document.getElementById("dynamic-fields");
                
                    // Clear existing fields
                    fields.innerHTML = "";
                
                    // Add fields based on category
                    if (category === "family") {
                        fields.innerHTML += '<label for="relationship">Relationship:</label>';
                        fields.innerHTML += '<input type="text" name="category_details[relationship]" placeholder="e.g., Brother"><br>';
                    } else if (category === "work") {
                        fields.innerHTML += '<label for="company">Company:</label>';
                        fields.innerHTML += '<input type="text" name="category_details[company]" placeholder="e.g., ABC Corp"><br>';
                        fields.innerHTML += '<label for="position">Position:</label>';
                        fields.innerHTML += '<input type="text" name="category_details[position]" placeholder="e.g., Manager"><br>';
                        fields.innerHTML += '<label for="department">Department:</label>';
                        fields.innerHTML += '<input type="text" name="category_details[department]" placeholder="e.g., Sales"><br>';
                    } else if (category === "school") {
                        fields.innerHTML += '<label for="institution">Institution:</label>';
                        fields.innerHTML += '<input type="text" name="category_details[institution]" placeholder="e.g., XYZ University"><br>';
                        fields.innerHTML += '<label for="level">Level:</label>';
                        fields.innerHTML += '<input type="text" name="category_details[level]" placeholder="e.g., BSc"><br>';
                        fields.innerHTML += '<label for="major">Major:</label>';
                        fields.innerHTML += '<input type="text" name="category_details[major]" placeholder="e.g., Computer Science"><br>';
                        fields.innerHTML += '<label for="grad_year">Graduation Year:</label>';
                        fields.innerHTML += '<input type="text" name="category_details[grad_year]" placeholder="e.g., 2024"><br>';
                    }
                }
                    
                function formatDate(inputId) {
                    const input = document.getElementById(inputId);
                    let dateValue = input.value;
                    if (dateValue) {
                        // Convert the date format from DD/MM/YYYY to YYYY-MM-DD
                        const dateParts = dateValue.split('/');
                        if (dateParts.length === 3) {
                            const formattedDate = `${dateParts[2]}-${dateParts[1]}-${dateParts[0]}`;
                            input.value = formattedDate;
                        }
                    }
                }

                // Add event listener to the date fields to apply the format when submitting the form
                document.getElementById("addForm").onsubmit = function() {
                    formatDate("dob");
                    formatDate("last_met");
                    formatDate("reminder");
                };

                document.getElementById("editForm").onsubmit = function() {
                    formatDate("dob");
                    formatDate("last_met");
                    formatDate("reminder");
                };
            """)

    # Create basic HTML templates if they don't exist
    if not os.path.exists("templates/index.html"):
        with open("templates/index.html", "w") as f:
            f.write("""\
                <!DOCTYPE html>
                <html lang="en">
                <head>
                    <meta charset="UTF-8">
                    <meta name="viewport" content="width=device-width, initial-scale=1.0">
                    <title>Welcome to Your Personal CRM</title>
                    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
                </head>
                <body>
                    <button id="toggleButton">Toggle Dark/Light Mode</button>
                    <h1>Welcome to Your Personal CRM</h1>
                    <nav>
                        <a href="/add">Add New Person</a> |
                        <a href="/view">View All Contacts</a> |
                        <a href="/reminders">View Reminders</a> |
                        <a href="/graph">View Relationship Graph</a> |
                        <a href="/edit/0">Profile</a>
                    </nav>

                    <h1>Do you want to generate a test data set?</h1>
                    <form method="post">
                        <button type="submit" name="choice" value="yes">Yes</button>
                        <button type="submit" name="choice" value="no">No</button>
                    </form>
                <script src="{{ url_for('static', filename='scripts.js') }}"></script>
                </body>
                </html>
            """)

    if not os.path.exists("templates/add_person.html"):
        with open("templates/add_person.html", "w") as f:
            f.write("""\
                    <!DOCTYPE html>
                    <html lang="en">
                    <head>
                        <meta charset="UTF-8">
                        <meta name="viewport" content="width=device-width, initial-scale=1.0">
                        <title>Add New Person</title>
                        <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
                    </head>
                    <body>
                        <header>
                            <button id="toggleButton">Toggle Dark/Light Mode</button>
                            <h1>Add a New Person</h1>
                            <nav>
                                <a href="/add">Add New Person</a> |
                                <a href="/view">View All Contacts</a> |
                                <a href="/reminders">View Reminders</a> |
                                <a href="/graph">View Relationship Graph</a> |
                                <a href="/edit/0">Profile</a>
                            </nav>
                            <hr>
                        </header>
                        <main>
                            <h1>Add New Person</h1>
                            <form method="POST" id="addForm">
                                {% if error %}
                                    <p style="color: red;">Error: {{ error }}</p>
                                {% endif %}
                                
                                <label for="first_name">First Name:</label>
                                <input type="text" id="first_name" name="first_name" required><br>

                                <label for="last_name">Last Name:</label>
                                <input type="text" id="last_name" name="last_name"><br>

                                <label for="category">Category:</label>
                                <select id="category" name="category" onchange="updateCategoryFields()">
                                    <option value="family">Family</option>
                                    <option value="work">Work</option>
                                    <option value="school">School</option>
                                    <option value="other">Other</option>
                                </select><br>

                                <!-- Dynamic fields placeholder -->
                                <div id="dynamic-fields"></div>

                                <label for="dob">Date of Birth:</label>
                                <input type="text" id="dob" name="dob" placeholder="DD/MM/YYYY"><br>

                                <label for="location">Location:</label>
                                <input type="text" id="location" name="location"><br>

                                <label for="last_met">Last Met:</label>
                                <input type="text" id="last_met" name="last_met" placeholder="DD/MM/YYYY"><br>

                                <label for="reminder">Reminder:</label>
                                <input type="text" id="reminder" name="reminder" placeholder="DD/MM/YYYY"><br>

                                <label for="loves">Loves (comma-separated):</label>
                                <input type="text" id="loves" name="loves"><br>

                                <label for="likes">Likes (comma-separated):</label>
                                <input type="text" id="likes" name="likes"><br>

                                <label for="dislikes">Dislikes (comma-separated):</label>
                                <input type="text" id="dislikes" name="dislikes"><br>

                                <label for="hates">Hates (comma-separated):</label>
                                <input type="text" id="hates" name="hates"><br>

                                <label for="connections">Connections:</label>
                                <select id="connections" name="connections" multiple>
                                    {% for person in existing_people %}
                                        <option value="{{ person['id'] }}">{{ person['first_name'] }} {{ person['last_name'] }}</option>
                                    {% endfor %}
                                </select><br><br>

                                <button type="submit">Save</button>
                            </form>
                            <a href="/">Back to Home</a>
                        </main>
                        <script src="{{ url_for('static', filename='scripts.js') }}"></script>
                        <script>
                            // Ensure dynamic fields update based on category
                            function updateCategoryFields() {
                                const category = document.getElementById("category").value.toLowerCase();
                                const fieldsContainer = document.getElementById("dynamic-fields");
                                fieldsContainer.innerHTML = ""; // Clear existing fields

                                if (category === "family") {
                                    fieldsContainer.innerHTML += `
                                        <label for="relationship">Relationship:</label>
                                        <input type="text" id="relationship" name="relationship" placeholder="e.g., Brother"><br>
                                    `;
                                } else if (category === "work") {
                                    fieldsContainer.innerHTML += `
                                        <label for="company">Company:</label>
                                        <input type="text" id="company" name="company" placeholder="e.g., ABC Corp"><br>
                                        <label for="position">Position:</label>
                                        <input type="text" id="position" name="position" placeholder="e.g., Manager"><br>
                                    `;
                                } else if (category === "school") {
                                    fieldsContainer.innerHTML += `
                                        <label for="institution">Institution:</label>
                                        <input type="text" id="institution" name="institution" placeholder="e.g., XYZ University"><br>
                                        <label for="level">Level:</label>
                                        <input type="text" id="level" name="level" placeholder="e.g., BSc"><br>
                                    `;
                                }
                            }
                            document.addEventListener("DOMContentLoaded", updateCategoryFields);
                        </script>
                    </body>
                    </html>
            """)

    # Create 'view_all_contacts.html' template
    if not os.path.exists("templates/view_all_contacts.html"):
        with open("templates/view_all_contacts.html", "w") as f:
            f.write("""\
                    <!DOCTYPE html>
                    <html lang="en">
                    <head>
                        <meta charset="UTF-8">
                        <meta name="viewport" content="width=device-width, initial-scale=1.0">
                        <title>All Contacts</title>
                        <link rel="stylesheet" href="/static/styles.css">
                    </head>
                    <body>
                        <header>
                            <button id="toggleButton">Toggle Dark/Light Mode</button>
                            <h1>All Contacts</h1>
                            <nav>
                                <a href="/add">Add New Person</a> |
                                <a href="/view">View All Contacts</a> |
                                <a href="/reminders">View Reminders</a> |
                                <a href="/graph">View Relationship Graph</a> |
                                <a href="/edit/0">Profile</a>
                            </nav>
                            <hr>
                        </header>

                        <!-- Search bar -->
                        <form method="get" action="/view">
                            <input type="text" name="search" placeholder="Search..." value="{{ search }}">
                            <button type="submit">Search</button>
                        </form>

                        {% if people %}
                            <!-- Table Headers with Sorting Links -->
                            <table>
                                <thead>
                                    <tr>
                                        <th><a href="/view?sort_by=id&order={{ 'desc' if sort_by == 'id' and order == 'asc' else 'asc' }}">ID</a></th>
                                        <th><a href="/view?sort_by=first_name&order={{ 'desc' if sort_by == 'first_name' and order == 'asc' else 'asc' }}">Name</a></th>
                                        <th><a href="/view?sort_by=category&order={{ 'desc' if sort_by == 'category' and order == 'asc' else 'asc' }}">Category</a></th>
                                        <th><a href="/view?sort_by=dob&order={{ 'desc' if sort_by == 'dob' and order == 'asc' else 'asc' }}">Date of Birth</a></th>
                                        <th>Location</th>
                                        <th><a href="/view?sort_by=last_met&order={{ 'desc' if sort_by == 'last_met' and order == 'asc' else 'asc' }}">Last Met</a></th>
                                        <th><a href="/view?sort_by=reminder&order={{ 'desc' if sort_by == 'reminder' and order == 'asc' else 'asc' }}">Reminder</a></th>
                                        <th>Loves</th>
                                        <th>Likes</th>
                                        <th>Dislikes</th>
                                        <th>Hates</th>
                                        <th>Connections</th>
                                        <th>Actions</th>
                                    </tr>
                                </thead>
                                <tbody>
                                    {% for person in people %}
                                    <tr>
                                        <td>{{ person.id }}</td>
                                        <td>{{ person.first_name }} {{ person.last_name }}</td>
                                        <td>{{ person.category }}</td>
                                        <td>{{ person.dob }}</td>
                                        <td>{{ person.location }}</td>
                                        <td>{{ person.last_met }}</td>
                                        <td>{{ person.reminder }}</td>
                                        <td>{{ person.personal_details.loves | join(", ") }}</td>
                                        <td>{{ person.personal_details.likes | join(", ") }}</td>
                                        <td>{{ person.personal_details.dislikes | join(", ") }}</td>
                                        <td>{{ person.personal_details.hates | join(", ") }}</td>
                                        <td>{{ person.connections | join(", ") }}</td>
                                        <td>
                                            <form action="{{ url_for('edit', person_id=person.id) }}" method="get">
                                                <button type="submit">Edit</button>
                                            </form>
                                            <form action="{{ url_for('delete', person_id=person.id) }}" method="post">
                                                <button type="submit">Delete</button>
                                            </form>
                                        </td>
                                    </tr>
                                    {% endfor %}
                                </tbody>
                            </table>
                            
                            <!-- Pagination Links -->
                            <div>
                                {% if pagination.has_prev %}
                                    <a href="/view?page={{ pagination.prev_page }}&sort_by={{ sort_by }}&order={{ order }}&search={{ search }}">Previous</a>
                                {% endif %}
                                Page {{ pagination.current_page }} of {{ pagination.total_pages }}
                                {% if pagination.has_next %}
                                    <a href="/view?page={{ pagination.next_page }}&sort_by={{ sort_by }}&order={{ order }}&search={{ search }}">Next</a>
                                {% endif %}
                            </div>
                        {% else %}
                            <p>No contacts available</p>
                        {% endif %}

                        <a href="/">Back to Home</a>
                    <script src="{{ url_for('static', filename='scripts.js') }}"></script>
                    </body>
                    </html>
            """)

    # Create 'edit_person.html' template
        if not os.path.exists("templates/edit_person.html"):
            with open("templates/edit_person.html", "w") as f:
                f.write("""\
                        <!DOCTYPE html>
                        <html lang="en">
                        <head>
                            <meta charset="UTF-8">
                            <meta name="viewport" content="width=device-width, initial-scale=1.0">
                            <title>Edit Person</title>
                            <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
                        </head>
                        <body>
                            <header>
                                <button id="toggleButton">Toggle Dark/Light Mode</button>
                                <h1>Edit an Existing Person</h1>
                                <nav>
                                    <a href="/add">Add New Person</a> |
                                    <a href="/view">View All Contacts</a> |
                                    <a href="/reminders">View Reminders</a> |
                                    <a href="/graph">View Relationship Graph</a> |
                                    <a href="/edit/0">Profile</a>
                                </nav>
                                <hr>
                            </header>
                            <h1>Edit Person</h1>
                            <form method="POST">
                                <label for="first_name">First Name:</label>
                                <input type="text" name="first_name" value="{{ person['first_name'] }}" required><br>

                                <label for="last_name">Last Name:</label>
                                <input type="text" name="last_name" value="{{ person['last_name'] }}"><br>

                                <label for="category">Category:</label>
                                <select id="category" name="category" onchange="updateCategoryFields()">
                                    <option value="family" {% if person and person['category']['name'] == 'family' %}selected{% endif %}>Family</option>
                                    <option value="work" {% if person and person['category']['name'] == 'work' %}selected{% endif %}>Work</option>
                                    <option value="school" {% if person and person['category']['name'] == 'school' %}selected{% endif %}>School</option>
                                    <option value="other" {% if person and person['category']['name'] == 'other' %}selected{% endif %}>Other</option>
                                </select><br>

                                <!-- Dynamic fields placeholder -->
                                <div id="dynamic-fields"></div>

                                <!-- Trigger dynamic field updates on page load for editing -->
                                <script>
                                    document.addEventListener("DOMContentLoaded", function () {
                                        updateCategoryFields();
                                    });
                                </script>

                                <label for="dob">Date of Birth:</label>
                                <input type="text" id="dob" name="dob" value="{{ person['dob'] if person else '' }}" placeholder="DD/MM/YYYY"><br>

                                <label for="location">Location:</label>
                                <input type="text" name="location"><br>

                                <label for="last_met">Last Met:</label>
                                <input type="text" id="last_met" name="last_met" value="{{ person['last_met'] if person else '' }}" placeholder="DD/MM/YYYY"><br>

                                <label for="reminder">Reminder:</label>
                                <input type="text" id="reminder" name="reminder" value="{{ person['reminder'] if person else '' }}" placeholder="DD/MM/YYYY"><br>

                                <label for="loves">Loves (comma-separated):</label>
                                <input type="text" name="loves" value="{{ ', '.join(person['personal_details'].get('loves', [])) }}"><br>

                                <label for="likes">Likes (comma-separated):</label>
                                <input type="text" name="likes" value="{{ ', '.join(person['personal_details'].get('likes', [])) }}"><br>

                                <label for="dislikes">Dislikes (comma-separated):</label>
                                <input type="text" name="dislikes" value="{{ ', '.join(person['personal_details'].get('dislikes', [])) }}"><br>

                                <label for="hates">Hates (comma-separated):</label>
                                <input type="text" name="hates" value="{{ ', '.join(person['personal_details'].get('hates', [])) }}"><br>

                                <label for="notes">Notes (comma-separated):</label>
                                <input type="text" name="notes" value="{{ ', '.join(person['personal_details'].get('notes', [])) }}"><br>

                                <label for="connections">Connections:</label>
                                <select name="connections" multiple>
                                    {% for p in data['people'] %}
                                        {% if p['id'] != person['id'] %}
                                            <option value="{{ p['id'] }}" 
                                                {% if p['id'] in person['connections'] or p['id'] == 0 %}selected{% endif %}>
                                                {{ p['first_name'] }} {{ p['last_name'] }}
                                            </option>
                                        {% endif %}
                                    {% endfor %}
                                </select><br>

                                <button type="submit">Save</button>
                            </form>
                                                
                            <br><a href="/view">Back to All Contacts</a>
                        <script src="{{ url_for('static', filename='scripts.js') }}"></script>
                        </body>
                        </html>
                """)

    # Create 'graph.html' template
    if not os.path.exists("templates/graph.html"):
        with open("templates/graph.html", "w") as f:
            f.write("""\
                    <!DOCTYPE html>
                    <html lang="en">
                    <head>
                        <meta charset="UTF-8">
                        <meta name="viewport" content="width=device-width, initial-scale=1.0">
                        <title>Relationship Graph</title>
                        <link rel="stylesheet" href="/static/styles.css">
                    </head>
                    <body>
                        <header>
                            <button id="toggleButton">Toggle Dark/Light Mode</button>
                            <h1>Interactive Relationship Graph</h1>
                            <nav>
                                <a href="/add">Add New Person</a> |
                                <a href="/view">View All Contacts</a> |
                                <a href="/reminders">View Reminders</a> |
                                <a href="/graph">View Relationship Graph</a> |
                                <a href="/edit/0">Profile</a>
                            </nav>
                            <hr>
                        </header>
                    
                        <!-- Graph Regeneration Button -->
                        <form method="post" action="/graph">
                            <button type="submit">Regenerate Graph</button>
                        </form>

                        <!-- Display Graph -->
                        <div>
                            <iframe src="{{ url_for('static', filename='graph.html') }}" width="100%" height="600px"></iframe>
                        </div>

                        <!-- Legend for Node Colors -->
                        <h3>Legend:</h3>
                        <table border="1">
                            <tr>
                                <td style="background-color: green; width: 20px;"></td>
                                <td>Met in the last week</td>
                            </tr>
                            <tr>
                                <td style="background-color: yellow; width: 20px;"></td>
                                <td>Met in the last month</td>
                            </tr>
                            <tr>
                                <td style="background-color: orange; width: 20px;"></td>
                                <td>Met in the last 3 months</td>
                            </tr>
                            <tr>
                                <td style="background-color: red; width: 20px;"></td>
                                <td>Met more than 3 months ago</td>
                            </tr>
                            <tr>
                                <td style="background-color: gray; width: 20px;"></td>
                                <td>No meeting data available</td>
                            </tr>
                        </table>

                        <a href="/">Back to Home</a>
                    <script src="{{ url_for('static', filename='scripts.js') }}"></script>
                    </body>
                    </html>
            """)

    # Create 'overdue_reminders.html' template for overdue reminders page
    if not os.path.exists("templates/overdue_reminders.html"):
        with open("templates/overdue_reminders.html", "w") as f:
            f.write("""\
                <!DOCTYPE html>
                <html lang="en">
                <head>
                    <meta charset="UTF-8">
                    <meta name="viewport" content="width=device-width, initial-scale=1.0">
                    <title>Overdue Reminders</title>
                    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
                </head>
                <body>
                    <header>
                        <button id="toggleButton">Toggle Dark/Light Mode</button>
                        <h1>Overdue Reminders</h1>
                        <nav>
                            <a href="/add">Add New Person</a> |
                            <a href="/view">View All Contacts</a> |
                            <a href="/reminders">View Reminders</a> |
                            <a href="/graph">View Relationship Graph</a> |
                            <a href="/edit/0">Profile</a>
                        </nav>
                        <hr>
                    </header>
                    <h1>Overdue Reminders</h1>
                    <table border="1">
                        <tr>
                            <th>Name</th>
                            <th>Category</th>
                            <th>Reminder</th>
                        </tr>
                        {% for person in overdue %}
                        <tr>
                            <td>{{ person['first_name'] }} {{ person.get('last_name', '') }}</td>
                            <td>{{ person['category'] }}</td>
                            <td>{{ person['reminder'] }}</td>
                        </tr>
                        {% endfor %}
                    </table>
                    <br><a href="/">Back to Home</a>
                    <script src="{{ url_for('static', filename='scripts.js') }}"></script>
                </body>
                </html>
            """)


    # Create 'no_overdue.html' template for no overdue reminders message
    if not os.path.exists("templates/no_overdue.html"):
        with open("templates/no_overdue.html", "w") as f:
            f.write("""\
                <!DOCTYPE html>
                <html lang="en">
                <head>
                    <meta charset="UTF-8">
                    <meta name="viewport" content="width=device-width, initial-scale=1.0">
                    <title>No Overdue Reminders</title>
                    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
                </head>
                <body>
                    <header>
                        <button id="toggleButton">Toggle Dark/Light Mode</button>
                        <h1>No Overdue Reminder</h1>
                        <nav>
                            <a href="/add">Add New Person</a> |
                            <a href="/view">View All Contacts</a> |
                            <a href="/reminders">View Reminders</a> |
                            <a href="/graph">View Relationship Graph</a> |
                            <a href="/edit/0">Profile</a>
                        </nav>
                        <hr>
                    </header>
                    <h1>No overdue reminders</h1>
                    <a href="/">Back to Home</a>
                    <script src="{{ url_for('static', filename='scripts.js') }}"></script>
                </body>
                </html>
            """)


# Initialize folder and file structure
initialize_folder_structure()

# 4. OOP consepts Initialization
class Category:
    def __init__(self, name):
        self.name = name

    def to_dict(self):
        """Convert category object to a dictionary."""
        return {"name": self.name}


class Family(Category):
    def __init__(self, name, relationship):
        super().__init__(name)
        self.relationship = relationship

    def to_dict(self):
        data = super().to_dict()
        data["relationship"] = self.relationship
        return data


class Work(Category):
    def __init__(self, name, company, position, department):
        super().__init__(name)
        self.company = company
        self.position = position
        self.department = department

    def to_dict(self):
        data = super().to_dict()
        data.update({
            "company": self.company,
            "position": self.position,
            "department": self.department,
        })
        return data


class School(Category):
    def __init__(self, name, institution, level, major, grad_year):
        super().__init__(name)
        self.institution = institution
        self.level = level
        self.major = major
        self.grad_year = grad_year

    def to_dict(self):
        data = super().to_dict()
        data.update({
            "institution": self.institution,
            "level": self.level,
            "major": self.major,
            "grad_year": self.grad_year,
        })
        return data
    
#data validation
def validate_non_empty(value, field_name):
    if not value.strip():
        raise ValueError(f"{field_name} cannot be empty.")
    return value.strip()

def validate_category(value):
    valid_categories = ["Family", "Work", "School", "Other"]
    if value.capitalize() not in valid_categories:
        raise ValueError(f"Invalid category '{value}'. Valid categories are: {', '.join(valid_categories)}.")
    return value.capitalize()

def validate_date(value, field_name):
    if not value.strip():
        return None  # Allow empty dates
    try:
        return parse_date(value)  # Assumes parse_date already handles formats
    except Exception:
        raise ValueError(f"{field_name} must be in the format DD/MM/YYYY.")


def validate_connections(value):
    try:
        return [int(conn_id) for conn_id in value]
    except ValueError:
        raise ValueError("Connections must be a list of valid integers.")


# 5. JSON Database Initialization & Helper Functions for JSON Operations
def initialize_json_file():
    if not os.path.exists("data.json"):
        with open("data.json", "w") as f:
            json.dump({"people": []}, f, indent=4)

# Initialize the JSON file if it doesn't exist
initialize_json_file()

# Load data from the JSON file
def load_data(file_name="data.json"):
    try:
        with open(file_name, "r") as f:
            data = json.load(f)
    except FileNotFoundError:
        print("Error: data.json file not found!")
        data = {"people": []}
    except json.JSONDecodeError:
        print("Error: Failed to decode JSON!")
        data = {"people": []}
    
    # Ensure the "You" profile (ID = 0) exists
    if not any(person['id'] == 0 for person in data['people']):
        data['people'].append({
            'id': 0,
            'first_name': 'You',
            'last_name': 'Profile',
            'category': 'Default',
            'dob': '',
            'location': '',
            'last_met': '',
            'reminder': '',
            'personal_details': {
                'loves': [],
                'likes': [],
                'dislikes': [],
                'hates': [],
                'notes': []
            },
            'connections': []
        })
        save_data(data)  # Save the data again with the new "You" profile
    
    # Ensure that every person (except ID = 0) has a connection to "You" (ID = 0)
    for person in data['people']:
        if person['id'] != 0 and 0 not in person['connections']:
            person['connections'].append(0)

    save_data(data)  # Ensure connections are saved with the default connection
    
    return data

# Save data to the JSON file
def save_data(data):
    def convert_dates(obj):
        if isinstance(obj, dict):
            return {key: convert_dates(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_dates(item) for item in obj]
        elif isinstance(obj, datetime):
            return obj.strftime("%d/%m/%Y")  # Convert datetime to string
        else:
            return obj

    # Convert dates before saving
    cleaned_data = convert_dates(data)

    with open("data.json", "w") as f:
        json.dump(cleaned_data, f, indent=4)

def add_to_undo_stack(action, data):
    if len(undo_stack) >= 5:
        undo_stack.pop(0)  # Remove the oldest action to keep only the last 5
    undo_stack.append({"action": action, "data": data})

def undo_last_action():
    if undo_stack:
        last_action = undo_stack.pop()
        action = last_action["action"]
        previous_data = last_action["data"]

        data = load_data()

        if action == "add":
            data["people"].remove(previous_data)
        elif action == "delete":
            data["people"].append(previous_data)
        elif action == "edit":
            person = next((p for p in data["people"] if p["id"] == previous_data["id"]), None)
            if person:
                person.update(previous_data)

        save_data(data)
        return redirect('/')

    return "No actions to undo.", 400

# Auto-generate unique ID for new entries
def generate_id(data):
    return len(data["people"]) + 1

# Add a new person to the JSON file
def add_person(data, first_name, last_name="", category_name="", dob="", location="", 
               last_met="", reminder="", category_details=None, loves=None, likes=None, dislikes=None, 
               hates=None, notes=None, connections=None):
    # Create category object based on category_name
    category = None
    if category_name.lower() == "family":
        category = Family(category_name, relationship=category_details.get("relationship", ""))
    elif category_name.lower() == "work":
        category = Work(
            category_name,
            company=category_details.get("company", ""),
            position=category_details.get("position", ""),
            department=category_details.get("department", ""),
        )
    elif category_name.lower() == "school":
        category = School(
            category_name,
            institution=category_details.get("institution", ""),
            level=category_details.get("level", ""),
            major=category_details.get("major", ""),
            grad_year=category_details.get("grad_year", ""),
        )
    else:
        category = Category(category_name)

    new_person = {
        "id": generate_id(data),
        "first_name": first_name,
        "last_name": last_name,
        "category": category,
        "dob": dob,
        "location": location,
        "last_met": last_met,
        "reminder": reminder,
        "personal_details": {
            "loves": loves or [],
            "likes": likes or [],
            "dislikes": dislikes or [],
            "hates": hates or []
        },
        "notes": notes or [],
        "connections": connections or []
    }
    data["people"].append(new_person)
    save_data(data)

# Find a person by their unique ID
def find_person_by_id(data, person_id):
    for person in data["people"]:
        if person["id"] == person_id:
            return person
    return None

# Update a person's details
def update_person(data, person_id, updated_fields, personal_details):
    person = find_person_by_id(data, person_id)
    
    if person:
        # Update basic fields
        for key, value in updated_fields.items():
            if key in person:
                person[key] = value
        
        # Update personal details (e.g., loves, likes, etc.)
        if "personal_details" not in person:
            person["personal_details"] = {}  # Initialize personal details if they don't exist
        
        for key, value in personal_details.items():
            if key in person["personal_details"]:
                person["personal_details"][key] = value
            else:
                person["personal_details"][key] = value
        
        # Ensure that ID 0 is always in the connections list (if not already present)
        if 0 not in person['connections']:
            person['connections'].append(0)

        save_data(data)
        return True
    return False

def parse_date(date_str):
    date_formats = ["%d/%m/%Y", "%Y-%m-%d"]  # Supports both formats
    for date_format in date_formats:
        try:
            return datetime.strptime(date_str, date_format)
        except (ValueError, TypeError):
            continue
    return None  # Return None if all formats fail


# Generate the relationship graph

#method2
def generate_graph_with_last_met(data):
    nodes = []
    edges = []
    colors = []  # Color list to store node colors based on the last_met date
    
    G = nx.Graph()  # Create a new NetworkX graph
    
    # Extract nodes (people) and edges (connections)
    for person in data.get("people", []):
        nodes.append(person)
        G.add_node(person["id"], label=f'{person["first_name"]} {person["last_name"]}')  # Add node with label
        for connection in person.get("connections", []):
            edges.append((person["id"], connection))
            G.add_edge(person["id"], connection)  # Add edge
    
    # Assign colors to nodes based on "last_met" field
    today = datetime.now()
    for person in nodes:
        last_met = person.get("last_met", "")
        if last_met:
            last_met_date = parse_date(last_met)
            if last_met_date:
                days_since = (today - last_met_date).days
                if days_since <= 7:
                    colors.append("green")  # < 1 week
                elif days_since <= 30:
                    colors.append("yellow")  # < 1 month
                elif days_since <= 90:
                    colors.append("orange")  # < 3 months
                else:
                    colors.append("red")  # > 3 months
            else:
                colors.append("gray")  # Invalid date
        else:
            colors.append("gray")  # No data
    
    # Use NetworkX to calculate the layout (positions) for nodes
    pos = nx.spring_layout(G, seed=42, k=0.5, iterations=50)  # Force-directed layout
    
    # Ensure that id=0 is always at the center
    if 0 in pos:
        pos[0] = (0, 0)  # Manually set the position of id=0 to (0, 0)
    
    # Get x and y positions for each node
    x_nodes = [pos[node][0] for node in G.nodes()]
    y_nodes = [pos[node][1] for node in G.nodes()]

    # Create Plotly figure for the graph
    fig = go.Figure()

    # Add edges to the graph
    for edge in edges:
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        fig.add_trace(go.Scatter(
            x=[x0, x1],
            y=[y0, y1],
            mode="lines",
            line=dict(width=1, color="gray"),
            hoverinfo="none"
        ))

    # Add nodes to the graph
    fig.add_trace(go.Scatter(
        x=x_nodes,
        y=y_nodes,
        mode="markers+text",
        marker=dict(size=10, color=colors),
        text=[f'{node["first_name"]} {node["last_name"]}' for node in nodes],
        hoverinfo="text"
    ))

    # Layout settings for the graph
    fig.update_layout(
        title="Relationship Graph (Color-coded by Last Met)",
        showlegend=False,
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False),
        plot_bgcolor="white",
        height=800,  # Adjust height to make the graph more visible
        width=1600    # Adjust width to make the graph more visible
    )

    # Save the graph to an HTML file
    fig.write_html("static/graph.html")

    return "static/graph.html"

def generate_test_data():

    def generate_name():
        nouns = ["Apple", "Banana", "Cherry", "Dragon", "Eagle", "Falcon", "Grape", "Honey", "Ivy", "Jasmine", 
                 "Kiwi", "Lemon", "Maple", "Nectar", "Olive", "Pear", "Quince", "Rose", "Sunflower", "Tulip"]
        first_name = random.choice(nouns)
        last_name = f"{first_name}son"
        return first_name, last_name

    def generate_date(start_year=1980, end_year=2024):
        start_date = datetime(start_year, 1, 1)
        end_date = datetime(end_year, 12, 31)
        random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
        return random_date.strftime("%d/%m/%Y")

    def generate_connections(current_id, total_ids):
        connections = {0}  # Ensure everyone is connected to ID 0
        max_connections = random.randint(1, 10)
        while len(connections) < max_connections:
            random_connection = random.randint(0, total_ids - 1)
            if random_connection != current_id:
                connections.add(random_connection)
        return list(connections)

    categories = ["Family", "Work", "School", "Other"]
    data = {"people": []}

    # Add central user
    data["people"].append({
        "id": 0,
        "first_name": "You",
        "last_name": "Profile",
        "category": "Default",
        "dob": "",
        "location": "",
        "last_met": "",
        "reminder": "",
        "personal_details": {
            "loves": [],
            "likes": [],
            "dislikes": [],
            "hates": [],
            "notes": []
        },
        "connections": list(range(1, 100))  # Connected to everyone else
    })

    # Add 99 more people
    for i in range(1, 100):
        first_name, last_name = generate_name()
        category = random.choice(categories)
        dob = generate_date(1950, 2020)
        last_met = generate_date(2023, 2024)
        reminder = generate_date(2023, 2025)
        connections = generate_connections(i, 100)
        personal_details = {
            "loves": [random.choice(["Pizza", "Books", "Music", "Movies", "Nature"])],
            "likes": [random.choice(["Cats", "Dogs", "Travel", "Art", "Cooking"])],
            "dislikes": [random.choice(["Noise", "Crowds", "Spicy Food", "Rain", "Traffic"])],
            "hates": [random.choice(["Lies", "Cold Weather", "Pollution", "Laziness", "Rudeness"])],
            "notes": [random.choice(["Met during vacation", "Colleague", "Childhood friend", "Neighbor", "Classmate"])]
        }

        data["people"].append({
            "id": i,
            "first_name": first_name,
            "last_name": last_name,
            "category": category,
            "dob": dob,
            "location": random.choice(["", "City A", "City B", "City C"]),
            "last_met": last_met,
            "reminder": reminder,
            "personal_details": personal_details,
            "connections": connections
        })

    test_file_path = "test_data.json"
    with open(test_file_path, "w") as f:
        json.dump(data, f, indent=4)
    return test_file_path


# 6. Flask Routes

# Home Page
@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        choice = request.form.get("choice")
        if choice == "yes":
            # Generate the test dataset
            test_file_path = generate_test_data()
            return redirect("/view?file=test")  # Pass a query parameter to indicate test data
        elif choice == "no":
            return redirect("/view?file=normal")  # Use the normal dataset
    return render_template("index.html")

# Add New Person Page
@app.route("/add", methods=["GET", "POST"])
def add():
    data = load_data()
    if request.method == "POST":
        try:
            # Validate and process form data
            first_name = validate_non_empty(request.form["first_name"], "First Name")
            last_name = request.form.get("last_name", "").strip()
            category = validate_category(request.form["category"])
            dob = validate_date(request.form["dob"], "Date of Birth")
            location = request.form.get("location", "").strip()
            last_met = validate_date(request.form["last_met"], "Last Met Date")
            reminder = validate_date(request.form["reminder"], "Reminder Date")

            # Parse personal details
            loves = request.form.get("loves", "").split(",") if request.form.get("loves") else []
            likes = request.form.get("likes", "").split(",") if request.form.get("likes") else []
            dislikes = request.form.get("dislikes", "").split(",") if request.form.get("dislikes") else []
            hates = request.form.get("hates", "").split(",") if request.form.get("hates") else []

            # Parse and validate connections
            selected_connections = request.form.getlist("connections")
            connections = validate_connections(selected_connections)

            # Create the new person entry
            new_person = {
                'id': len(data['people']),
                'first_name': first_name,
                'last_name': last_name,
                'category': category,
                "dob": dob.strftime("%d/%m/%Y") if dob else "",
                'location': location,
                "last_met": last_met.strftime("%d/%m/%Y") if last_met else "",
                "reminder": reminder.strftime("%d/%m/%Y") if reminder else "",
                'personal_details': {
                    'loves': loves,
                    'likes': likes,
                    'dislikes': dislikes,
                    'hates': hates,
                    'notes': []
                },
                'connections': connections
            }

            # Ensure the new person is connected to ID 0 and vice versa
            if 0 not in connections:
                connections.append(0)

            # Update mutual connections
            for friend_id in connections:
                friend = find_person_by_id(data, friend_id)
                if friend:
                    if new_person["id"] not in friend["connections"]:
                        friend["connections"].append(new_person["id"])
                    if friend_id not in new_person["connections"]:
                        new_person["connections"].append(friend_id)

            # Append the new person and save
            data['people'].append(new_person)
            save_data(data)
            return redirect("/")

        except ValueError as e:
            # Return validation error to the user
            existing_people = data["people"]
            return render_template("add_person.html", existing_people=existing_people, error=str(e))

    # Pass existing people for connections dropdown
    existing_people = data["people"]
    return render_template("add_person.html", existing_people=existing_people, error=None)


@app.route("/edit/<int:person_id>", methods=["GET", "POST"])
def edit(person_id):
    data = load_data()
    person = find_person_by_id(data, person_id)

    if not person:
        return "Person not found", 404

    if request.method == "POST":
        try:
            # Validate and process form data
            first_name = validate_non_empty(request.form["first_name"], "First Name")
            last_name = request.form.get("last_name", "").strip()
            category = validate_category(request.form["category"])
            dob = validate_date(request.form["dob"], "Date of Birth")
            location = request.form.get("location", "").strip()
            last_met = validate_date(request.form["last_met"], "Last Met Date")
            reminder = validate_date(request.form["reminder"], "Reminder Date")

            # Parse personal details
            personal_details = {
                "loves": request.form.get("loves", "").split(","),
                "likes": request.form.get("likes", "").split(","),
                "dislikes": request.form.get("dislikes", "").split(","),
                "hates": request.form.get("hates", "").split(","),
                "notes": request.form.get("notes", "").split(",")
            }

            # Parse and validate connections
            selected_connections = request.form.getlist("connections")
            connections = validate_connections(selected_connections)

            # Ensure ID 0 is in the connections list
            if 0 not in connections:
                connections.append(0)

            # Update the person's data
            person.update({
                "first_name": first_name,
                "last_name": last_name,
                "category": category,
                "dob": dob.strftime("%d/%m/%Y") if dob else "",
                "location": location,
                "last_met": last_met.strftime("%d/%m/%Y") if last_met else "",
                "reminder": reminder.strftime("%d/%m/%Y") if reminder else "",
                "connections": connections,
                "personal_details": personal_details
            })

            # Save the updated data
            save_data(data)

            # Redirect appropriately
            if person_id == 0:
                return redirect("/edit/0")
            return redirect("/view")

        except ValueError as e:
            # Return validation error to the user
            return render_template("edit_person.html", person=person, data=data, error=str(e))

    return render_template("edit_person.html", person=person, data=data, error=None)

@app.route("/delete/<int:person_id>", methods=["POST"])
def delete(person_id):
    data = load_data()
    person_to_delete = next((p for p in data["people"] if p["id"] == person_id), None)

    if person_to_delete:
        # Add the delete action to the undo stack
        add_to_undo_stack("delete", person_to_delete)

        # Remove the person from the list
        data["people"] = [p for p in data["people"] if p["id"] != person_id]
        save_data(data)
        return redirect('/')
    
    return "Person not found", 404

@app.route("/undo", methods=["POST"]) 
def undo_route():
    return undo_last_action()

# View All Contacts Page
@app.route("/view")
def view():
    file_type = request.args.get("file", "normal")  # Default to normal
    if file_type == "test":
        data = load_data("test_data.json")
    else:
        data = load_data("data.json")

    people = data.get("people", [])

    # Get query parameters for sorting, searching, and pagination
    sort_by = request.args.get("sort_by", "id")  # Default sorting by "id"
    order = request.args.get("order", "asc")  # Default order is ascending
    search = request.args.get("search", "").lower()  # Search keyword
    page = int(request.args.get("page", 1))  # Default to page 1
    per_page = int(request.args.get("per_page", 100))  # Default 10 records per page

    # Helper function to parse dates (returns None if invalid)
    def parse_date(date_str):
        try:
            return datetime.strptime(date_str, "%d/%m/%Y")
        except (ValueError, TypeError):
            return None

    # Apply search filtering
    if search:
        people = [
            person for person in people
            if search in person["first_name"].lower()
            or search in person["last_name"].lower()
            or search in person["category"].lower()
        ]

    # Apply sorting (handle dates specifically)
    reverse = (order == "desc")
    if sort_by in {"dob", "last_met", "reminder"}:
        people = sorted(people, key=lambda x: parse_date(x.get(sort_by, "")) or datetime.min, reverse=reverse)
    else:
        people = sorted(people, key=lambda x: x.get(sort_by, ""), reverse=reverse)

    # Apply pagination
    total = len(people)
    start = (page - 1) * per_page
    end = start + per_page
    people = people[start:end]

    # Pass pagination details to the template
    pagination = {
        "current_page": page,
        "total_pages": (total + per_page - 1) // per_page,  # Ceiling division
        "has_next": end < total,
        "has_prev": start > 0,
        "next_page": page + 1,
        "prev_page": page - 1
    }

    return render_template(
        "view_all_contacts.html",
        people=people,
        sort_by=sort_by,
        order=order,
        search=search,
        pagination=pagination
    )

# View Overdue Reminders Page
@app.route("/reminders")
def reminders():
    data = load_data()  # Ensure this loads your data correctly
    today = datetime.now()
    overdue = []

    # Helper function to parse dates
    def parse_reminder_date(date_str):
        try:
            return datetime.strptime(date_str, "%d/%m/%Y")  # Try DD/MM/YYYY first
        except ValueError:
            try:
                return datetime.strptime(date_str, "%Y-%m-%d")  # Fallback to YYYY-MM-DD
            except ValueError:
                return None  # Invalid date

    # Check for overdue reminders
    for person in data["people"]:
        if person["reminder"]:
            reminder_date = parse_reminder_date(person["reminder"])
            if reminder_date and reminder_date < today:
                overdue.append(person)

    # Render the appropriate template based on the presence of overdue reminders
    if overdue:
        # If there are overdue reminders, render the 'overdue_reminders.html' template
        return render_template('overdue_reminders.html', overdue=overdue)
    else:
        # If there are no overdue reminders, render the 'no_overdue.html' template
        return render_template('no_overdue.html')

@app.route("/graph", methods=["GET", "POST"])
def graph():
    if request.method == "POST":
        # Regenerate the graph when the button is clicked
        graph_file = generate_graph_with_last_met(load_data())
        return render_template("graph.html", graph_file=graph_file, regenerate=True)

    # Initially generate the graph when the page is loaded
    graph_file = generate_graph_with_last_met(load_data())
    return render_template("graph.html", graph_file=graph_file, regenerate=False)


# Run the Flask application
if __name__ == "__main__":
    app.run()
'''


In [8]:
import subprocess

# Save the project code to a temporary file (project.py)
with open("project.py", "w") as f:
    f.write(project_code)

# Start the Flask app in a separate process using subprocess
flask_process = subprocess.Popen(["python", "project.py"])

In [9]:
import os
import signal
while True:
    user_input = input("Now you can interact with the application at: HTTP://http://127.0.0.1:5000/. Type 'STOP' to shut down the Flask app: ")
    if user_input.strip().upper() == "STOP":
        print("Shutting down Flask server...")
        flask_process.terminate()  # Terminate the Flask subprocess
        break

Shutting down Flask server...


In [10]:
print("done")

done
